In [1]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

In [2]:
BASE = Path("../data/RDD2020")
IMG_DIRS = {
    "train": BASE / "images/train",
    "val": BASE / "images/val",
}
ANN_DIRS = {
    "train": BASE / "annotations/train",
    "val": BASE / "annotations/val",
}
LBL_DIRS = {
    "train": BASE / "labels/train",
    "val": BASE / "labels/val",
}

for p in LBL_DIRS.values():
    p.mkdir(parents=True, exist_ok=True)


In [ ]:
def voc_to_yolo_bbox(bbox, img_w, img_h):
    xmin, ymin, xmax, ymax = bbox
    xc = ((xmin + xmax) / 2) / img_w
    yc = ((ymin + ymax) / 2) / img_h
    w = (xmax - xmin) / img_w
    h = (ymax - ymin) / img_h
    return xc, yc, w, h


In [4]:
POTHOLE_CLASS = "D40"
CLASS_ID = 0

def convert_split(split):
    ann_dir = ANN_DIRS[split]
    lbl_dir = LBL_DIRS[split]

    for xml_file in ann_dir.glob("*.xml"):
        tree = ET.parse(xml_file)
        root = tree.getroot()

        size = root.find("size")
        img_w = int(size.find("width").text)
        img_h = int(size.find("height").text)

        yolo_lines = []

        for obj in root.findall("object"):
            name = obj.find("name").text
            if name != POTHOLE_CLASS:
                continue

            bnd = obj.find("bndbox")
            bbox = (
                float(bnd.find("xmin").text),
                float(bnd.find("ymin").text),
                float(bnd.find("xmax").text),
                float(bnd.find("ymax").text),
            )

            xc, yc, w, h = voc_to_yolo_bbox(bbox, img_w, img_h)
            yolo_lines.append(f"{CLASS_ID} {xc} {yc} {w} {h}")

        if yolo_lines:
            out_file = lbl_dir / f"{xml_file.stem}.txt"
            out_file.write_text("\n".join(yolo_lines))


In [5]:
convert_split("train")
convert_split("val")


In [6]:
print("Train labels:", len(list(LBL_DIRS["train"].glob("*.txt"))))
print("Val labels:", len(list(LBL_DIRS["val"].glob("*.txt"))))


Train labels: 1530
Val labels: 0


In [7]:
import xml.etree.ElementTree as ET

def has_d40(xml_path):
    root = ET.parse(xml_path).getroot()
    return any(obj.find("name").text == "D40" for obj in root.findall("object"))


In [8]:
d40_xmls = [
    x for x in ANN_DIRS["train"].glob("*.xml")
    if has_d40(x)
]

len(d40_xmls)


1530

In [9]:
import random
random.seed(42)

val_count = int(0.1 * len(d40_xmls))
val_xmls = random.sample(d40_xmls, val_count)

for xml in val_xmls:
    img = BASE / "images/train" / (xml.stem + ".jpg")

    xml.rename(BASE / "annotations/val" / xml.name)
    img.rename(BASE / "images/val" / img.name)


In [10]:
convert_split("val")


In [11]:
print("Train labels:", len(list(LBL_DIRS["train"].glob("*.txt"))))
print("Val labels:", len(list(LBL_DIRS["val"].glob("*.txt"))))


Train labels: 1530
Val labels: 153
